# Node Parameters

Up to this point we've dealt with defining nodes and composing them together into pipelines. One issue is that all of the logic is static, the pipelines are unable to changed by external sources. Node Parameters are the solution to this- they comprise named arguments of a node that are defined at the point where the pipeline is run. 

## Definition

In [ ]:
import pandas as pd
from flypipe.node import node

@node(type="pandas")
def t1(param1):
    print(f"param1: `{param1}`")
    return pd.DataFrame()

@node(
    type="pandas",
    dependencies=[t1]
)
def t2(t1):
    return t1
    
df = t2.run(parameters={t1: {"param1": "hello"}})

Note just like any python function argument we can give a default value to use if no value is provided at runtime. 

In [ ]:
import pandas as pd
from flypipe.node import node

@node(type="pandas")
def t1(param1=None):
    print(f"param1: `{param1}`")
    return pd.DataFrame()
    
df = t1()

## Parameters and node functions

Parameters can also be used with node functions, together you have more flexibility on dynamically changing not only node behaviours but also graph compositions

In [ ]:
from flypipe import node_function

@node_function()
def t1_func(node_selector="t1"):
    
    @node(type="pandas")
    def t1():
        return pd.DataFrame()
    
    if node_selector == "t1":
        return t1
    
    @node(
        type="pandas",
        dependencies=[t1]
    )
    def t2():
        return pd.DataFrame()
    
    return t2, t1

displayHTML(t1_func.html())    

#### Changing parameter `node_selector`

In [ ]:

displayHTML(
    t1_func.html(
        parameters={
            t1_func: {'node_selector': 't2'}
        }
    ))

## Suggested use cases for node parameters

- you need the node to behave differently accordingly to some settings
- train a model inside a node with changeable splits of data between train, test and validation
- run nodes that execute customisable versions of an mlflow model by encoding the model run id into a node parameter


**train a model with different splits of data (train, test and validation)**

In [ ]:
from flypipe.node import node
from flypipe.schema import Schema, Column
from flypipe.schema.types import Date
import pandas as pd
from datetime import datetime, timedelta

@node(
    type="pandas",
    output=Schema([
        Column("date", Date(), "date"),
    ])
)
def my_model_features():
    
    date = datetime(2022,1,1)
    dates = [date + timedelta(days=day) for day in range(11)]
    
    return pd.DataFrame(data={"date": dates})

@node(
    type="pandas",
    dependencies=[my_model_features]
    
)
def tag_data(my_model_features, split={'train': datetime(2022,1,4), "test": datetime(2022,1,7)}):
    print(f"""
    Split data
    ----------
    train: date <= {split['train'].date()}
    test: {split['train'].date()} < date <= {split['test'].date()}
    val: {split['test'].date()} < date
    """)
    
    my_model_features["tag"] = "val"
    my_model_features.loc[my_model_features['date'] <= split['test'], "tag"] = "test"
    my_model_features.loc[my_model_features['date'] <= split['train'], "tag"] = "train"    
    
    return my_model_features

@node(
    type="pandas",
    dependencies=[tag_data]
    
)
def my_model(tag_data):    
    return tag_data

df = my_model.run()
display(df)

**running with different splits**

In [ ]:
df = my_model.run(parameters={
     tag_data: {
         'split': {'train': datetime(2022,1,2), "test": datetime(2022,1,4)}
     }
})
display(df)